In [ ]:
Iceberg

In [1]:
%%configure -f
{
  "conf": {
    "spark.sql.extensions": "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "spark.sql.catalog.dev": "org.apache.iceberg.spark.SparkCatalog",
    "spark.sql.catalog.dev.warehouse": "s3://apacheiceberg/iceberg11",
    "spark.sql.catalog.dev.catalog-impl": "org.apache.iceberg.aws.glue.GlueCatalog",
    "spark.sql.catalog.dev.io-impl": "org.apache.iceberg.aws.s3.S3FileIO",
    "spark.sql.catalog.dev.lock-impl": "org.apache.iceberg.aws.dynamodb.DynamoDbLockManager",
    "spark.sql.catalog.dev.lock.table": "myGlueLockTable"
  }
}

In [2]:
db = 'dev.iceberg100'
db_path = 's3://apacheiceberg/iceberg100/'
raw_data = 's3:dev.iceberg100.customer_address.ca_state //dwh100/data/'
table_format = 'iceberg'

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1705348621891_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
db = 'dev.iceberg11'
db_path = 's3://apacheiceberg/iceberg11/'
raw_data = 's3://raw1tcpds/data/'
table_format = 'iceberg'

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1705620742334_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Check if the database exists
existing_databases = spark.sql("SHOW DATABASES")
existing_databases_list = [row["namespace"] for row in existing_databases.collect()]

if db not in existing_databases_list:
    # If the database does not exist, create it
    spark.sql(f"CREATE DATABASE {db} LOCATION '{db_path}'")
    print(f"Database '{db}' created.")
else:
    # If the database already exists, print a message
    print(f"Database '{db}' already exists.")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
[SCHEMA_ALREADY_EXISTS] Cannot create schema `iceberg100` because it already exists.
Choose a different name, drop the existing schema, or add the IF NOT EXISTS clause to tolerate pre-existing schema.
Traceback (most recent call last):
  File "/mnt/yarn/usercache/livy/appcache/application_1705346256285_0003/container_1705346256285_0003_01_000001/pyspark.zip/pyspark/sql/session.py", line 1440, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery, litArgs), self)
  File "/mnt/yarn/usercache/livy/appcache/application_1705346256285_0003/container_1705346256285_0003_01_000001/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1323, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/mnt/yarn/usercache/livy/appcache/application_1705346256285_0003/container_1705346256285_0003_01_000001/pyspark.zip/pyspark/errors/exceptions/captured.py", line 175, in deco
    raise converted from None
pyspark.errors.exceptions.captured.AnalysisExcep

In [3]:
execution_times = {}
import time

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()


table_name = 'store_sales'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    ss_sold_date_sk           integer                       ,
    ss_sold_time_sk           integer                       ,
    ss_item_sk                integer                          ,
    ss_customer_sk            integer                       ,
    ss_cdemo_sk               integer                       ,
    ss_hdemo_sk               integer                       ,
    ss_addr_sk                integer                       ,
    ss_store_sk               integer                       ,
    ss_promo_sk               integer                       ,
    ss_ticket_number          bigint                           ,
    ss_quantity               integer                       ,
    ss_wholesale_cost         decimal(7,2)                  ,
    ss_list_price             decimal(7,2)                  ,
    ss_sales_price            decimal(7,2)                  ,
    ss_ext_discount_amt       decimal(7,2)                  ,
    ss_ext_sales_price        decimal(7,2)                  ,
    ss_ext_wholesale_cost     decimal(7,2)                  ,
    ss_ext_list_price         decimal(7,2)                  ,
    ss_ext_tax                decimal(7,2)                  ,
    ss_coupon_amt             decimal(7,2)                  ,
    ss_net_paid               decimal(7,2)                  ,
    ss_net_paid_inc_tax       decimal(7,2)                  ,
    ss_net_profit             decimal(7,2)    
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])

# Define column names and types
columns_mapping = [
    ("ss_sold_date_sk", IntegerType()),
    ("ss_sold_time_sk", IntegerType()),
    ("ss_item_sk", IntegerType()),
    ("ss_customer_sk", IntegerType()),
    ("ss_cdemo_sk", IntegerType()),
    ("ss_hdemo_sk", IntegerType()),
    ("ss_addr_sk", IntegerType()),
    ("ss_store_sk", IntegerType()),
    ("ss_promo_sk", IntegerType()),
    ("ss_ticket_number", LongType()),
    ("ss_quantity", IntegerType()),
    ("ss_wholesale_cost", DecimalType(7,2)),
    ("ss_list_price", DecimalType(7,2)),
    ("ss_sales_price", DecimalType(7,2)),
    ("ss_ext_discount_amt", DecimalType(7,2)),
    ("ss_ext_sales_price", DecimalType(7,2)),
    ("ss_ext_wholesale_cost", DecimalType(7,2)),
    ("ss_ext_list_price", DecimalType(7,2)),
    ("ss_ext_tax", DecimalType(7,2)),
    ("ss_coupon_amt", DecimalType(7,2)),
    ("ss_net_paid", DecimalType(7,2)),
    ("ss_net_paid_inc_tax", DecimalType(7,2)),
    ("ss_net_profit", DecimalType(7,2))
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))
    
# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

    
# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time

# Store the execution time in a list with the variable name
execution_times[table_name] = execution_time

# Print the execution time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interrupted by user


In [5]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()


table_name = 'store_sales'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    ss_sold_date_sk           integer                       ,
    ss_sold_time_sk           integer                       ,
    ss_item_sk                integer                          ,
    ss_customer_sk            integer                       ,
    ss_cdemo_sk               integer                       ,
    ss_hdemo_sk               integer                       ,
    ss_addr_sk                integer                       ,
    ss_store_sk               integer                       ,
    ss_promo_sk               integer                       ,
    ss_ticket_number          bigint                           ,
    ss_quantity               integer                       ,
    ss_wholesale_cost         decimal(7,2)                  ,
    ss_list_price             decimal(7,2)                  ,
    ss_sales_price            decimal(7,2)                  ,
    ss_ext_discount_amt       decimal(7,2)                  ,
    ss_ext_sales_price        decimal(7,2)                  ,
    ss_ext_wholesale_cost     decimal(7,2)                  ,
    ss_ext_list_price         decimal(7,2)                  ,
    ss_ext_tax                decimal(7,2)                  ,
    ss_coupon_amt             decimal(7,2)                  ,
    ss_net_paid               decimal(7,2)                  ,
    ss_net_paid_inc_tax       decimal(7,2)                  ,
    ss_net_profit             decimal(7,2)    
)
USING {table_format} 
LOCATION '{table_path}'""");

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [6]:
# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])

df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----+-----+-----+-------+----+-----+---+----+---+----+-----+------+-----+-------+-------+-------+--------+------+-------+-------+-------+--------+
|    _c0|  _c1|  _c2|  _c3|    _c4| _c5|  _c6|_c7| _c8|_c9|_c10| _c11|  _c12| _c13|   _c14|   _c15|   _c16|    _c17|  _c18|   _c19|   _c20|   _c21|    _c22|
+-------+-----+-----+-----+-------+----+-----+---+----+---+----+-----+------+-----+-------+-------+-------+--------+------+-------+-------+-------+--------+
|2451813|65495| 3617|67006| 591617|3428|24839| 10| 161|  1|  79|11.41| 18.71| 2.80|  99.54| 221.20| 901.39| 1478.09|  6.08|  99.54| 121.66| 127.74| -779.73|
|2451813|65495|13283|67006| 591617|3428|24839| 10| 154|  1|  37|63.63|101.17|41.47|  46.03|1534.39|2354.31| 3743.29| 59.53|  46.03|1488.36|1547.89| -865.95|
|2451813|65495|13631|67006| 591617|3428|24839| 10| 172|  1|  99|80.52|137.68|83.98|   0.00|8314.02|7971.48|13630.32|  0.00|   0.00|8314.02|8314.02|  342.54|
|2451813|65495| 5981|67006| 591617|3428|24839| 10| 280|  1

In [8]:
# Define column names and types
columns_mapping = [
    ("ss_sold_date_sk", IntegerType()),
    ("ss_sold_time_sk", IntegerType()),
    ("ss_item_sk", IntegerType()),
    ("ss_customer_sk", IntegerType()),
    ("ss_cdemo_sk", IntegerType()),
    ("ss_hdemo_sk", IntegerType()),
    ("ss_addr_sk", IntegerType()),
    ("ss_store_sk", IntegerType()),
    ("ss_promo_sk", IntegerType()),
    ("ss_ticket_number", LongType()),
    ("ss_quantity", IntegerType()),
    ("ss_wholesale_cost", DecimalType(7,2)),
    ("ss_list_price", DecimalType(7,2)),
    ("ss_sales_price", DecimalType(7,2)),
    ("ss_ext_discount_amt", DecimalType(7,2)),
    ("ss_ext_sales_price", DecimalType(7,2)),
    ("ss_ext_wholesale_cost", DecimalType(7,2)),
    ("ss_ext_list_price", DecimalType(7,2)),
    ("ss_ext_tax", DecimalType(7,2)),
    ("ss_coupon_amt", DecimalType(7,2)),
    ("ss_net_paid", DecimalType(7,2)),
    ("ss_net_paid_inc_tax", DecimalType(7,2)),
    ("ss_net_profit", DecimalType(7,2))
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))
    
# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

    
# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time

# Store the execution time in a list with the variable name
execution_times[table_name] = execution_time

# Print the execution time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for store_sales: 94.32084965705872 seconds

In [19]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'catalog_sales'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    cs_sold_date_sk           integer                       ,
    cs_sold_time_sk           integer                       ,
    cs_ship_date_sk           integer                       ,
    cs_bill_customer_sk       integer                       ,
    cs_bill_cdemo_sk          integer                       ,
    cs_bill_hdemo_sk          integer                       ,
    cs_bill_addr_sk           integer                       ,
    cs_ship_customer_sk       integer                       ,
    cs_ship_cdemo_sk          integer                       ,
    cs_ship_hdemo_sk          integer                       ,
    cs_ship_addr_sk           integer                       ,
    cs_call_center_sk         integer                       ,
    cs_catalog_page_sk        integer                       ,
    cs_ship_mode_sk           integer                       ,
    cs_warehouse_sk           integer                       ,
    cs_item_sk                integer                       ,
    cs_promo_sk               integer                       ,
    cs_order_number           bigint                        ,
    cs_quantity               integer                       ,
    cs_wholesale_cost         decimal(7,2)                  ,
    cs_list_price             decimal(7,2)                  ,
    cs_sales_price            decimal(7,2)                  ,
    cs_ext_discount_amt       decimal(7,2)                  ,
    cs_ext_sales_price        decimal(7,2)                  ,
    cs_ext_wholesale_cost     decimal(7,2)                  ,
    cs_ext_list_price         decimal(7,2)                  ,
    cs_ext_tax                decimal(7,2)                  ,
    cs_coupon_amt             decimal(7,2)                  ,
    cs_ext_ship_cost          decimal(7,2)                  ,
    cs_net_paid               decimal(7,2)                  ,
    cs_net_paid_inc_tax       decimal(7,2)                  ,
    cs_net_paid_inc_ship      decimal(7,2)                  ,
    cs_net_paid_inc_ship_tax  decimal(7,2)                  ,
    cs_net_profit             decimal(7,2)       
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("cs_sold_date_sk", IntegerType()),
    ("cs_sold_time_sk", IntegerType()),
    ("cs_ship_date_sk", IntegerType()),
    ("cs_bill_customer_sk", IntegerType()),
    ("cs_bill_cdemo_sk", IntegerType()),
    ("cs_bill_hdemo_sk", IntegerType()),
    ("cs_bill_addr_sk", IntegerType()),
    ("cs_ship_customer_sk", IntegerType()),
    ("cs_ship_cdemo_sk", IntegerType()),
    ("cs_ship_hdemo_sk", IntegerType()),
    ("cs_ship_addr_sk", IntegerType()),
    ("cs_call_center_sk", IntegerType()),
    ("cs_catalog_page_sk", IntegerType()),
    ("cs_ship_mode_sk", IntegerType()),
    ("cs_warehouse_sk", IntegerType()),
    ("cs_item_sk", IntegerType()),
    ("cs_promo_sk", IntegerType()),
    ("cs_order_number", LongType()),
    ("cs_quantity", IntegerType()),
    ("cs_wholesale_cost", DecimalType(7, 2)),
    ("cs_list_price", DecimalType(7, 2)),
    ("cs_sales_price", DecimalType(7, 2)),
    ("cs_ext_discount_amt", DecimalType(7, 2)),
    ("cs_ext_sales_price", DecimalType(7, 2)),
    ("cs_ext_wholesale_cost", DecimalType(7, 2)),
    ("cs_ext_list_price", DecimalType(7, 2)),
    ("cs_ext_tax", DecimalType(7, 2)),
    ("cs_coupon_amt", DecimalType(7, 2)),
    ("cs_ext_ship_cost", DecimalType(7, 2)),
    ("cs_net_paid", DecimalType(7, 2)),
    ("cs_net_paid_inc_tax", DecimalType(7, 2)),
    ("cs_net_paid_inc_ship", DecimalType(7, 2)),
    ("cs_net_paid_inc_ship_tax", DecimalType(7, 2)),
    ("cs_net_profit", DecimalType(7, 2)),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for catalog_sales: 35.23167896270752 seconds

In [26]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'web_sales'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    ws_sold_date_sk           integer                       ,
    ws_sold_time_sk           integer                       ,
    ws_ship_date_sk           integer                       ,
    ws_item_sk                integer                       ,
    ws_bill_customer_sk       integer                       ,
    ws_bill_cdemo_sk          integer                       ,
    ws_bill_hdemo_sk          integer                       ,
    ws_bill_addr_sk           integer                       ,
    ws_ship_customer_sk       integer                       ,
    ws_ship_cdemo_sk          integer                       ,
    ws_ship_hdemo_sk          integer                       ,
    ws_ship_addr_sk           integer                       ,
    ws_web_page_sk            integer                       ,
    ws_web_site_sk            integer                       ,
    ws_ship_mode_sk           integer                       ,
    ws_warehouse_sk           integer                       ,
    ws_promo_sk               integer                       ,
    ws_order_number           bigint                        ,
    ws_quantity               integer                       ,
    ws_wholesale_cost         decimal(7,2)                  ,
    ws_list_price             decimal(7,2)                  ,
    ws_sales_price            decimal(7,2)                  ,
    ws_ext_discount_amt       decimal(7,2)                  ,
    ws_ext_sales_price        decimal(7,2)                  ,
    ws_ext_wholesale_cost     decimal(7,2)                  ,
    ws_ext_list_price         decimal(7,2)                  ,
    ws_ext_tax                decimal(7,2)                  ,
    ws_coupon_amt             decimal(7,2)                  ,
    ws_ext_ship_cost          decimal(7,2)                  ,
    ws_net_paid               decimal(7,2)                  ,
    ws_net_paid_inc_tax       decimal(7,2)                  ,
    ws_net_paid_inc_ship      decimal(7,2)                  ,
    ws_net_paid_inc_ship_tax  decimal(7,2)                  ,
    ws_net_profit             decimal(7,2)  
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("ws_sold_date_sk", IntegerType()),
    ("ws_sold_time_sk", IntegerType()),
    ("ws_ship_date_sk", IntegerType()),
    ("ws_bill_customer_sk", IntegerType()),
    ("ws_bill_cdemo_sk", IntegerType()),
    ("ws_bill_hdemo_sk", IntegerType()),
    ("ws_bill_addr_sk", IntegerType()),
    ("ws_ship_customer_sk", IntegerType()),
    ("ws_ship_cdemo_sk", IntegerType()),
    ("ws_ship_hdemo_sk", IntegerType()),
    ("ws_ship_addr_sk", IntegerType()),
    ("ws_web_page_sk", IntegerType()),
    ("ws_web_site_sk", IntegerType()),
    ("ws_ship_mode_sk", IntegerType()),
    ("ws_warehouse_sk", IntegerType()),
    ("ws_item_sk", IntegerType()),
    ("ws_promo_sk", IntegerType()),
    ("ws_order_number", LongType()),
    ("ws_quantity", IntegerType()),
    ("ws_wholesale_cost", DecimalType(7, 2)),
    ("ws_list_price", DecimalType(7, 2)),
    ("ws_sales_price", DecimalType(7, 2)),
    ("ws_ext_discount_amt", DecimalType(7, 2)),
    ("ws_ext_sales_price", DecimalType(7, 2)),
    ("ws_ext_wholesale_cost", DecimalType(7, 2)),
    ("ws_ext_list_price", DecimalType(7, 2)),
    ("ws_ext_tax", DecimalType(7, 2)),
    ("ws_coupon_amt", DecimalType(7, 2)),
    ("ws_ext_ship_cost", DecimalType(7, 2)),
    ("ws_net_paid", DecimalType(7, 2)),
    ("ws_net_paid_inc_tax", DecimalType(7, 2)),
    ("ws_net_paid_inc_ship", DecimalType(7, 2)),
    ("ws_net_paid_inc_ship_tax", DecimalType(7, 2)),
    ("ws_net_profit", DecimalType(7, 2)),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for web_sales: 11.114461660385132 seconds

In [28]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'store_returns'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    sr_returned_date_sk       integer                       ,
    sr_return_time_sk         integer                       ,
    sr_item_sk                integer                          ,
    sr_customer_sk            integer                       ,
    sr_cdemo_sk               integer                       ,
    sr_hdemo_sk               integer                       ,
    sr_addr_sk                integer                       ,
    sr_store_sk               integer                       ,
    sr_reason_sk              integer                       ,
    sr_ticket_number          bigint                           ,
    sr_return_quantity        integer                       ,
    sr_return_amt             decimal(7,2)                  ,
    sr_return_tax             decimal(7,2)                  ,
    sr_return_amt_inc_tax     decimal(7,2)                  ,
    sr_fee                    decimal(7,2)                  ,
    sr_return_ship_cost       decimal(7,2)                  ,
    sr_refunded_cash          decimal(7,2)                  ,
    sr_reversed_charge        decimal(7,2)                  ,
    sr_store_credit           decimal(7,2)                  ,
    sr_net_loss               decimal(7,2)    
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("sr_returned_date_sk", IntegerType()),
    ("sr_return_time_sk", IntegerType()),
    ("sr_item_sk", IntegerType()),
    ("sr_customer_sk", IntegerType()),
    ("sr_cdemo_sk", IntegerType()),
    ("sr_hdemo_sk", IntegerType()),
    ("sr_addr_sk", IntegerType()),
    ("sr_store_sk", IntegerType()),
    ("sr_reason_sk", IntegerType()),
    ("sr_ticket_number", LongType()),
    ("sr_return_quantity", IntegerType()),
    ("sr_return_amt", DecimalType(7, 2)),
    ("sr_return_tax", DecimalType(7, 2)),
    ("sr_return_amt_inc_tax", DecimalType(7, 2)),
    ("sr_fee", DecimalType(7, 2)),
    ("sr_return_ship_cost", DecimalType(7, 2)),
    ("sr_refunded_cash", DecimalType(7, 2)),
    ("sr_reversed_charge", DecimalType(7, 2)),
    ("sr_store_credit", DecimalType(7, 2)),
    ("sr_net_loss", DecimalType(7, 2)),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for store_returns: 9.412431716918945 seconds

In [29]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'catalog_returns'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    cr_returned_date_sk       integer                       ,
    cr_returned_time_sk       integer                       ,
    cr_item_sk                integer                          ,
    cr_refunded_customer_sk   integer                       ,
    cr_refunded_cdemo_sk      integer                       ,
    cr_refunded_hdemo_sk      integer                       ,
    cr_refunded_addr_sk       integer                       ,
    cr_returning_customer_sk  integer                       ,
    cr_returning_cdemo_sk     integer                       ,
    cr_returning_hdemo_sk     integer                       ,
    cr_returning_addr_sk      integer                       ,
    cr_call_center_sk         integer                       ,
    cr_catalog_page_sk        integer                       ,
    cr_ship_mode_sk           integer                       ,
    cr_warehouse_sk           integer                       ,
    cr_reason_sk              integer                       ,
    cr_order_number           bigint                           ,
    cr_return_quantity        integer                       ,
    cr_return_amount          decimal(7,2)                  ,
    cr_return_tax             decimal(7,2)                  ,
    cr_return_amt_inc_tax     decimal(7,2)                  ,
    cr_fee                    decimal(7,2)                  ,
    cr_return_ship_cost       decimal(7,2)                  ,
    cr_refunded_cash          decimal(7,2)                  ,
    cr_reversed_charge        decimal(7,2)                  ,
    cr_store_credit           decimal(7,2)                  ,
    cr_net_loss               decimal(7,2)
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("cr_returned_date_sk", IntegerType()),
    ("cr_returned_time_sk", IntegerType()),
    ("cr_item_sk", IntegerType()),
    ("cr_refunded_customer_sk", IntegerType()),
    ("cr_refunded_cdemo_sk", IntegerType()),
    ("cr_refunded_hdemo_sk", IntegerType()),
    ("cr_refunded_addr_sk", IntegerType()),
    ("cr_returning_customer_sk", IntegerType()),
    ("cr_returning_cdemo_sk", IntegerType()),
    ("cr_returning_hdemo_sk", IntegerType()),
    ("cr_returning_addr_sk", IntegerType()),
    ("cr_call_center_sk", IntegerType()),
    ("cr_catalog_page_sk", IntegerType()),
    ("cr_ship_mode_sk", IntegerType()),
    ("cr_warehouse_sk", IntegerType()),
    ("cr_reason_sk", IntegerType()),
    ("cr_order_number", LongType()),
    ("cr_return_quantity", IntegerType()),
    ("cr_return_amount", DecimalType(7, 2)),
    ("cr_return_tax", DecimalType(7, 2)),
    ("cr_return_amt_inc_tax", DecimalType(7, 2)),
    ("cr_fee", DecimalType(7, 2)),
    ("cr_return_ship_cost", DecimalType(7, 2)),
    ("cr_refunded_cash", DecimalType(7, 2)),
    ("cr_reversed_charge", DecimalType(7, 2)),
    ("cr_store_credit", DecimalType(7, 2)),
    ("cr_net_loss", DecimalType(7, 2)),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for catalog_returns: 5.483922481536865 seconds

In [62]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'inventory'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    inv_date_sk               integer                          ,
    inv_item_sk               integer                          ,
    inv_warehouse_sk          integer                          ,
    inv_quantity_on_hand      integer
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("inv_date_sk", IntegerType()),
    ("inv_item_sk", IntegerType()),
    ("inv_warehouse_sk", IntegerType()),
    ("inv_quantity_on_hand", IntegerType()),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for inventory: 20.63095498085022 seconds

In [30]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'web_returns'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    wr_returned_date_sk       integer                       ,
    wr_returned_time_sk       integer                       ,
    wr_item_sk                integer                          ,
    wr_refunded_customer_sk   integer                       ,
    wr_refunded_cdemo_sk      integer                       ,
    wr_refunded_hdemo_sk      integer                       ,
    wr_refunded_addr_sk       integer                       ,
    wr_returning_customer_sk  integer                       ,
    wr_returning_cdemo_sk     integer                       ,
    wr_returning_hdemo_sk     integer                       ,
    wr_returning_addr_sk      integer                       ,
    wr_web_page_sk            integer                       ,
    wr_reason_sk              integer                       ,
    wr_order_number           bigint                     ,
    wr_return_quantity        integer                       ,
    wr_return_amt             decimal(7,2)                  ,
    wr_return_tax             decimal(7,2)                  ,
    wr_return_amt_inc_tax     decimal(7,2)                  ,
    wr_fee                    decimal(7,2)                  ,
    wr_return_ship_cost       decimal(7,2)                  ,
    wr_refunded_cash          decimal(7,2)                  ,
    wr_reversed_charge        decimal(7,2)                  ,
    wr_account_credit         decimal(7,2)                  ,
    wr_net_loss               decimal(7,2) 
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("wr_returned_date_sk", IntegerType()),
    ("wr_returned_time_sk", IntegerType()),
    ("wr_item_sk", IntegerType()),
    ("wr_refunded_customer_sk", IntegerType()),
    ("wr_refunded_cdemo_sk", IntegerType()),
    ("wr_refunded_hdemo_sk", IntegerType()),
    ("wr_refunded_addr_sk", IntegerType()),
    ("wr_returning_customer_sk", IntegerType()),
    ("wr_returning_cdemo_sk", IntegerType()),
    ("wr_returning_hdemo_sk", IntegerType()),
    ("wr_returning_addr_sk", IntegerType()),
    ("wr_web_page_sk", IntegerType()),
    ("wr_reason_sk", IntegerType()),
    ("wr_order_number", LongType()),
    ("wr_return_quantity", IntegerType()),
    ("wr_return_amt", DecimalType(7,2)),
    ("wr_return_tax", DecimalType(7,2)),
    ("wr_return_amt_inc_tax", DecimalType(7,2)),
    ("wr_fee", DecimalType(7,2)),
    ("wr_return_ship_cost", DecimalType(7,2)),
    ("wr_refunded_cash", DecimalType(7,2)),
    ("wr_reversed_charge", DecimalType(7,2)),
    ("wr_account_credit", DecimalType(7,2)),
    ("wr_net_loss", DecimalType(7,2)),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for web_returns: 4.118833303451538 seconds

In [31]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'customer'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    c_customer_sk             integer                          ,
    c_customer_id             char(16)                         ,
    c_current_cdemo_sk        integer                       ,
    c_current_hdemo_sk        integer                       ,
    c_current_addr_sk         integer                       ,
    c_first_shipto_date_sk    integer                       ,
    c_first_sales_date_sk     integer                       ,
    c_salutation              char(10)                      ,
    c_first_name              char(20)                      ,
    c_last_name               char(30)                      ,
    c_preferred_cust_flag     char(1)                       ,
    c_birth_day               integer                       ,
    c_birth_month             integer                       ,
    c_birth_year              integer                       ,
    c_birth_country           varchar(20)                   ,
    c_login                   char(13)                      ,
    c_email_address           char(50)                      ,
    c_last_review_date_sk     integer 
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("c_customer_sk", IntegerType()),
    ("c_customer_id", StringType()),
    ("c_current_cdemo_sk", IntegerType()),
    ("c_current_hdemo_sk", IntegerType()),
    ("c_current_addr_sk", IntegerType()),
    ("c_first_shipto_date_sk", IntegerType()),
    ("c_first_sales_date_sk", IntegerType()),
    ("c_salutation", StringType()),
    ("c_first_name", StringType()),
    ("c_last_name", StringType()),
    ("c_preferred_cust_flag", StringType()),
    ("c_birth_day", IntegerType()),
    ("c_birth_month", IntegerType()),
    ("c_birth_year", IntegerType()),
    ("c_birth_country", StringType()),
    ("c_login", StringType()),
    ("c_email_address", StringType()),
    ("c_last_review_date_sk", IntegerType()),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for customer: 3.7857563495635986 seconds

In [32]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'customer_address'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    ca_address_sk             integer                          ,
    ca_address_id             char(16)                         ,
    ca_street_number          char(10)                      ,
    ca_street_name            varchar(60)                   ,
    ca_street_type            char(15)                      ,
    ca_suite_number           char(10)                      ,
    ca_city                   varchar(60)                   ,
    ca_county                 varchar(30)                   ,
    ca_state                  char(2)                       ,
    ca_zip                    char(10)                      ,
    ca_country                varchar(20)                   ,
    ca_gmt_offset             decimal(5,2)                  ,
    ca_location_type          char(20) 
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("ca_address_sk", IntegerType()),
    ("ca_address_id", StringType()),
    ("ca_street_number", StringType()),
    ("ca_street_name", StringType()),
    ("ca_street_type", StringType()),
    ("ca_suite_number", StringType()),
    ("ca_city", StringType()),
    ("ca_county", StringType()),
    ("ca_state", StringType()),
    ("ca_zip", StringType()),
    ("ca_country", StringType()),
    ("ca_gmt_offset", DecimalType(5, 2)),
    ("ca_location_type", StringType()),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for customer_address: 2.9811716079711914 seconds

In [33]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'item'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    i_item_sk                 integer                          ,
    i_item_id                 char(16)                         ,
    i_rec_start_date          date                          ,
    i_rec_end_date            date                          ,
    i_item_desc               varchar(200)                  ,
    i_current_price           decimal(7,2)                  ,
    i_wholesale_cost          decimal(7,2)                  ,
    i_brand_id                integer                       ,
    i_brand                   char(50)                      ,
    i_class_id                integer                       ,
    i_class                   char(50)                      ,
    i_category_id             integer                       ,
    i_category                char(50)                      ,
    i_manufact_id             integer                       ,
    i_manufact                char(50)                      ,
    i_size                    char(20)                      ,
    i_formulation             char(20)                      ,
    i_color                   char(20)                      ,
    i_units                   char(10)                      ,
    i_container               char(10)                      ,
    i_manager_id              integer                       ,
    i_product_name            char(50)   
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types

columns_mapping = [
    ("i_item_sk", IntegerType()),
    ("i_item_id", StringType()),
    ("i_rec_start_date", DateType()),
    ("i_rec_end_date", DateType()),
    ("i_item_desc", StringType()),
    ("i_current_price", DecimalType(7, 2)),
    ("i_wholesale_cost", DecimalType(7, 2)),
    ("i_brand_id", IntegerType()),
    ("i_brand", StringType()),
    ("i_class_id", IntegerType()),
    ("i_class", StringType()),
    ("i_category_id", IntegerType()),
    ("i_category", StringType()),
    ("i_manufact_id", IntegerType()),
    ("i_manufact", StringType()),
    ("i_size", StringType()),
    ("i_formulation", StringType()),
    ("i_color", StringType()),
    ("i_units", StringType()),
    ("i_container", StringType()),
    ("i_manager_id", IntegerType()),
    ("i_product_name", StringType()),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for item: 3.179847240447998 seconds

In [34]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'customer_demographics'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    cd_demo_sk                integer                          ,
    cd_gender                 char(1)                       ,
    cd_marital_status         char(1)                       ,
    cd_education_status       char(20)                      ,
    cd_purchase_estimate      integer                       ,
    cd_credit_rating          char(10)                      ,
    cd_dep_count              integer                       ,
    cd_dep_employed_count     integer                       ,
    cd_dep_college_count      integer   
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("cd_demo_sk", IntegerType()),
    ("cd_gender", StringType()),
    ("cd_marital_status", StringType()),
    ("cd_education_status", StringType()),
    ("cd_purchase_estimate", IntegerType()),
    ("cd_credit_rating", StringType()),
    ("cd_dep_count", IntegerType()),
    ("cd_dep_employed_count", IntegerType()),
    ("cd_dep_college_count", IntegerType()),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for customer_demographics: 6.831796884536743 seconds

In [35]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'date_dim'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    d_date_sk                 integer                          ,
    d_date_id                 char(16)                         ,
    d_date                    date                          ,
    d_month_seq               integer                       ,
    d_week_seq                integer                       ,
    d_quarter_seq             integer                       ,
    d_year                    integer                       ,
    d_dow                     integer                       ,
    d_moy                     integer                       ,
    d_dom                     integer                       ,
    d_qoy                     integer                       ,
    d_fy_year                 integer                       ,
    d_fy_quarter_seq          integer                       ,
    d_fy_week_seq             integer                       ,
    d_day_name                char(9)                       ,
    d_quarter_name            char(6)                       ,
    d_holiday                 char(1)                       ,
    d_weekend                 char(1)                       ,
    d_following_holiday       char(1)                       ,
    d_first_dom               integer                       ,
    d_last_dom                integer                       ,
    d_same_day_ly             integer                       ,
    d_same_day_lq             integer                       ,
    d_current_day             char(1)                       ,
    d_current_week            char(1)                       ,
    d_current_month           char(1)                       ,
    d_current_quarter         char(1)                       ,
    d_current_year            char(1)
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("d_date_sk", IntegerType()),
    ("d_date_id", StringType()),
    ("d_date", DateType()),
    ("d_month_seq", IntegerType()),
    ("d_week_seq", IntegerType()),
    ("d_quarter_seq", IntegerType()),
    ("d_year", IntegerType()),
    ("d_dow", IntegerType()),
    ("d_moy", IntegerType()),
    ("d_dom", IntegerType()),
    ("d_qoy", IntegerType()),
    ("d_fy_year", IntegerType()),
    ("d_fy_quarter_seq", IntegerType()),
    ("d_fy_week_seq", IntegerType()),
    ("d_day_name", StringType()),
    ("d_quarter_name", StringType()),
    ("d_holiday", StringType()),
    ("d_weekend", StringType()),
    ("d_following_holiday", StringType()),
    ("d_first_dom", IntegerType()),
    ("d_last_dom", IntegerType()),
    ("d_same_day_ly", IntegerType()),
    ("d_same_day_lq", IntegerType()),
    ("d_current_day", StringType()),
    ("d_current_week", StringType()),
    ("d_current_month", StringType()),
    ("d_current_quarter", StringType()),
    ("d_current_year", StringType()),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for date_dim: 3.951873779296875 seconds

In [36]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'time_dim'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    t_time_sk                 integer                          ,
    t_time_id                 char(16)                         ,
    t_time                    integer                       ,
    t_hour                    integer                       ,
    t_minute                  integer                       ,
    t_second                  integer                       ,
    t_am_pm                   char(2)                       ,
    t_shift                   char(20)                      ,
    t_sub_shift               char(20)                      ,
    t_meal_time               char(20)  
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("t_time_sk", IntegerType()),
    ("t_time_id", StringType()),
    ("t_time", IntegerType()),
    ("t_hour", IntegerType()),
    ("t_minute", IntegerType()),
    ("t_second", IntegerType()),
    ("t_am_pm", StringType()),
    ("t_shift", StringType()),
    ("t_sub_shift", StringType()),
    ("t_meal_time", StringType()),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for time_dim: 2.725959539413452 seconds

In [37]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'catalog_page'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    cp_catalog_page_sk        integer                          ,
    cp_catalog_page_id        char(16)                         ,
    cp_start_date_sk          integer                       ,
    cp_end_date_sk            integer                       ,
    cp_department             varchar(50)                   ,
    cp_catalog_number         integer                       ,
    cp_catalog_page_number    integer                       ,
    cp_description            varchar(100)                  ,
    cp_type                   varchar(100)   
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("cp_catalog_page_sk", IntegerType()),
    ("cp_catalog_page_id", StringType()),
    ("cp_start_date_sk", IntegerType()),
    ("cp_end_date_sk", IntegerType()),
    ("cp_department", StringType()),
    ("cp_catalog_number", IntegerType()),
    ("cp_catalog_page_number", IntegerType()),
    ("cp_description", StringType()),
    ("cp_type", StringType()),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for catalog_page: 2.17922043800354 seconds

In [38]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'web_page'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    wp_web_page_sk            integer                          ,
    wp_web_page_id            char(16)                         ,
    wp_rec_start_date         date                          ,
    wp_rec_end_date           date                          ,
    wp_creation_date_sk       integer                       ,
    wp_access_date_sk         integer                       ,
    wp_autogen_flag           char(1)                       ,
    wp_customer_sk            integer                       ,
    wp_url                    varchar(100)                  ,
    wp_type                   char(50)                      ,
    wp_char_count             integer                       ,
    wp_link_count             integer                       ,
    wp_image_count            integer                       ,
    wp_max_ad_count           integer  
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("wp_web_page_sk", IntegerType()),
    ("wp_web_page_id", StringType()),
    ("wp_rec_start_date", DateType()),
    ("wp_rec_end_date", DateType()),
    ("wp_creation_date_sk", IntegerType()),
    ("wp_access_date_sk", IntegerType()),
    ("wp_autogen_flag", StringType()),
    ("wp_customer_sk", IntegerType()),
    ("wp_url", StringType()),
    ("wp_type", StringType()),
    ("wp_char_count", IntegerType()),
    ("wp_link_count", IntegerType()),
    ("wp_image_count", IntegerType()),
    ("wp_max_ad_count", IntegerType()),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for web_page: 2.040485143661499 seconds

In [39]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'store'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    s_store_sk                integer                          ,
    s_store_id                char(16)                         ,
    s_rec_start_date          date                          ,
    s_rec_end_date            date                          ,
    s_closed_date_sk          integer                       ,
    s_store_name              varchar(50)                   ,
    s_number_employees        integer                       ,
    s_floor_space             integer                       ,
    s_hours                   char(20)                      ,
    s_manager                 varchar(40)                   ,
    s_market_id               integer                       ,
    s_geography_class         varchar(100)                  ,
    s_market_desc             varchar(100)                  ,
    s_market_manager          varchar(40)                   ,
    s_division_id             integer                       ,
    s_division_name           varchar(50)                   ,
    s_company_id              integer                       ,
    s_company_name            varchar(50)                   ,
    s_street_number           varchar(10)                   ,
    s_street_name             varchar(60)                   ,
    s_street_type             char(15)                      ,
    s_suite_number            char(10)                      ,
    s_city                    varchar(60)                   ,
    s_county                  varchar(30)                   ,
    s_state                   char(2)                       ,
    s_zip                     char(10)                      ,
    s_country                 varchar(20)                   ,
    s_gmt_offset              decimal(5,2)                  ,
    s_tax_precentage          decimal(5,2)  
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("s_store_sk", IntegerType()),
    ("s_store_id", StringType()),
    ("s_rec_start_date", DateType()),
    ("s_rec_end_date", DateType()),
    ("s_closed_date_sk", IntegerType()),
    ("s_store_name", StringType()),
    ("s_number_employees", IntegerType()),
    ("s_floor_space", IntegerType()),
    ("s_hours", StringType()),
    ("s_manager", StringType()),
    ("s_market_id", IntegerType()),
    ("s_geography_class", StringType()),
    ("s_market_desc", StringType()),
    ("s_market_manager", StringType()),
    ("s_division_id", IntegerType()),
    ("s_division_name", StringType()),
    ("s_company_id", IntegerType()),
    ("s_company_name", StringType()),
    ("s_street_number", StringType()),
    ("s_street_name", StringType()),
    ("s_street_type", StringType()),
    ("s_suite_number", StringType()),
    ("s_city", StringType()),
    ("s_county", StringType()),
    ("s_state", StringType()),
    ("s_zip", StringType()),
    ("s_country", StringType()),
    ("s_gmt_offset", DecimalType(5, 2)),
    ("s_tax_precentage", DecimalType(5, 2)),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for store: 2.6003503799438477 seconds

In [40]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'promotion'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    p_promo_sk                integer                          ,
    p_promo_id                char(16)                         ,
    p_start_date_sk           integer                       ,
    p_end_date_sk             integer                       ,
    p_item_sk                 integer                       ,
    p_cost                    decimal(15,2)                 ,
    p_response_target         integer                       ,
    p_promo_name              char(50)                      ,
    p_channel_dmail           char(1)                       ,
    p_channel_email           char(1)                       ,
    p_channel_catalog         char(1)                       ,
    p_channel_tv              char(1)                       ,
    p_channel_radio           char(1)                       ,
    p_channel_press           char(1)                       ,
    p_channel_event           char(1)                       ,
    p_channel_demo            char(1)                       ,
    p_channel_details         varchar(100)                  ,
    p_purpose                 char(15)                      ,
    p_discount_active         char(1)  
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types

columns_mapping = [
    ("p_promo_sk", IntegerType()),
    ("p_promo_id", StringType()),
    ("p_start_date_sk", IntegerType()),
    ("p_end_date_sk", IntegerType()),
    ("p_item_sk", IntegerType()),
    ("p_cost", DecimalType(15, 2)),
    ("p_response_target", IntegerType()),
    ("p_promo_name", StringType()),
    ("p_channel_dmail", StringType()),
    ("p_channel_email", StringType()),
    ("p_channel_catalog", StringType()),
    ("p_channel_tv", StringType()),
    ("p_channel_radio", StringType()),
    ("p_channel_press", StringType()),
    ("p_channel_event", StringType()),
    ("p_channel_demo", StringType()),
    ("p_channel_details", StringType()),
    ("p_purpose", StringType()),
    ("p_discount_active", StringType()),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for promotion: 2.164320468902588 seconds

In [41]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'household_demographics'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    hd_demo_sk                integer                          ,
    hd_income_band_sk         integer                       ,
    hd_buy_potential          char(15)                      ,
    hd_dep_count              integer                       ,
    hd_vehicle_count          integer  
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("hd_demo_sk", IntegerType()),
    ("hd_income_band_sk", IntegerType()),
    ("hd_buy_potential", StringType()),
    ("hd_dep_count", IntegerType()),
    ("hd_vehicle_count", IntegerType()),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for household_demographics: 1.8356091976165771 seconds

In [42]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'web_site'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    web_site_sk               integer                          ,
    web_site_id               char(16)                         ,
    web_rec_start_date        date                          ,
    web_rec_end_date          date                          ,
    web_name                  varchar(50)                   ,
    web_open_date_sk          integer                       ,
    web_close_date_sk         integer                       ,
    web_class                 varchar(50)                   ,
    web_manager               varchar(40)                   ,
    web_mkt_id                integer                       ,
    web_mkt_class             varchar(50)                   ,
    web_mkt_desc              varchar(100)                  ,
    web_market_manager        varchar(40)                   ,
    web_company_id            integer                       ,
    web_company_name          char(50)                      ,
    web_street_number         char(10)                      ,
    web_street_name           varchar(60)                   ,
    web_street_type           char(15)                      ,
    web_suite_number          char(10)                      ,
    web_city                  varchar(60)                   ,
    web_county                varchar(30)                   ,
    web_state                 char(2)                       ,
    web_zip                   char(10)                      ,
    web_country               varchar(20)                   ,
    web_gmt_offset            decimal(5,2)                  ,
    web_tax_percentage        decimal(5,2)   
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("web_site_sk", IntegerType()),
    ("web_site_id", StringType()),
    ("web_rec_start_date", DateType()),
    ("web_rec_end_date", DateType()),
    ("web_name", StringType()),
    ("web_open_date_sk", IntegerType()),
    ("web_close_date_sk", IntegerType()),
    ("web_class", StringType()),
    ("web_manager", StringType()),
    ("web_mkt_id", IntegerType()),
    ("web_mkt_class", StringType()),
    ("web_mkt_desc", StringType()),
    ("web_market_manager", StringType()),
    ("web_company_id", IntegerType()),
    ("web_company_name", StringType()),
    ("web_street_number", StringType()),
    ("web_street_name", StringType()),
    ("web_street_type", StringType()),
    ("web_suite_number", StringType()),
    ("web_city", StringType()),
    ("web_county", StringType()),
    ("web_state", StringType()),
    ("web_zip", StringType()),
    ("web_country", StringType()),
    ("web_gmt_offset", DecimalType(5, 2)),
    ("web_tax_percentage", DecimalType(5,2)),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for web_site: 3.0325467586517334 seconds

In [43]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'call_center'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    cc_call_center_sk         integer                          ,
    cc_call_center_id         char(16)                         ,
    cc_rec_start_date         date                          ,
    cc_rec_end_date           date                          ,
    cc_closed_date_sk         integer                       ,
    cc_open_date_sk           integer                       ,
    cc_name                   varchar(50)                   ,
    cc_class                  varchar(50)                   ,
    cc_employees              integer                       ,
    cc_sq_ft                  integer                       ,
    cc_hours                  char(20)                      ,
    cc_manager                varchar(40)                   ,
    cc_mkt_id                 integer                       ,
    cc_mkt_class              char(50)                      ,
    cc_mkt_desc               varchar(100)                  ,
    cc_market_manager         varchar(40)                   ,
    cc_division               integer                       ,
    cc_division_name          varchar(50)                   ,
    cc_company                integer                       ,
    cc_company_name           char(50)                      ,
    cc_street_number          char(10)                      ,
    cc_street_name            varchar(60)                   ,
    cc_street_type            char(15)                      ,
    cc_suite_number           char(10)                      ,
    cc_city                   varchar(60)                   ,
    cc_county                 varchar(30)                   ,
    cc_state                  char(2)                       ,
    cc_zip                    char(10)                      ,
    cc_country                varchar(20)                   ,
    cc_gmt_offset             decimal(5,2)                  ,
    cc_tax_percentage         decimal(5,2)  
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("cc_call_center_sk", IntegerType()),
    ("cc_call_center_id", StringType()),
    ("cc_rec_start_date", DateType()),
    ("cc_rec_end_date", DateType()),
    ("cc_closed_date_sk", IntegerType()),
    ("cc_open_date_sk", IntegerType()),
    ("cc_name", StringType()),
    ("cc_class", StringType()),
    ("cc_employees", IntegerType()),
    ("cc_sq_ft", IntegerType()),
    ("cc_hours", StringType()),
    ("cc_manager", StringType()),
    ("cc_mkt_id", IntegerType()),
    ("cc_mkt_class", StringType()),
    ("cc_mkt_desc", StringType()),
    ("cc_market_manager", StringType()),
    ("cc_division", IntegerType()),
    ("cc_division_name", StringType()),
    ("cc_company", IntegerType()),
    ("cc_company_name", StringType()),
    ("cc_street_number", StringType()),
    ("cc_street_name", StringType()),
    ("cc_street_type", StringType()),
    ("cc_suite_number", StringType()),
    ("cc_city", StringType()),
    ("cc_county", StringType()),
    ("cc_state", StringType()),
    ("cc_zip", StringType()),
    ("cc_country", StringType()),
    ("cc_gmt_offset", DecimalType(5, 2)),
    ("cc_tax_percentage", DecimalType(5, 2)),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for call_center: 2.7348875999450684 seconds

In [44]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'warehouse'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    w_warehouse_sk            integer                          ,
    w_warehouse_id            char(16)                         ,
    w_warehouse_name          varchar(20)                   ,
    w_warehouse_sq_ft         integer                       ,
    w_street_number           char(10)                      ,
    w_street_name             varchar(60)                   ,
    w_street_type             char(15)                      ,
    w_suite_number            char(10)                      ,
    w_city                    varchar(60)                   ,
    w_county                  varchar(30)                   ,
    w_state                   char(2)                       ,
    w_zip                     char(10)                      ,
    w_country                 varchar(20)                   ,
    w_gmt_offset              decimal(5,2)  
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("w_warehouse_sk", IntegerType()),
    ("w_warehouse_id", StringType()),
    ("w_warehouse_name", StringType()),
    ("w_warehouse_sq_ft", IntegerType()),
    ("w_street_number", StringType()),
    ("w_street_name", StringType()),
    ("w_street_type", StringType()),
    ("w_suite_number", StringType()),
    ("w_city", StringType()),
    ("w_county", StringType()),
    ("w_state", StringType()),
    ("w_zip", StringType()),
    ("w_country", StringType()),
    ("w_gmt_offset", DecimalType(5, 2)),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for warehouse: 1.9480865001678467 seconds

In [45]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'reason'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    r_reason_sk               integer                          ,
    r_reason_id               char(16)                         ,
    r_reason_desc             char(100)   
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("r_reason_sk", IntegerType()),
    ("r_reason_id", StringType()),
    ("r_reason_desc", StringType()),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for reason: 1.7782957553863525 seconds

In [46]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'ship_mode'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    sm_ship_mode_sk           integer                          ,
    sm_ship_mode_id           char(16)                         ,
    sm_type                   char(30)                      ,
    sm_code                   char(10)                      ,
    sm_carrier                char(20)                      ,
    sm_contract               char(20)  
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("sm_ship_mode_sk", IntegerType()),
    ("sm_ship_mode_id", StringType()),
    ("sm_type", StringType()),
    ("sm_code", StringType()),
    ("sm_carrier", StringType()),
    ("sm_contract", StringType()),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for ship_mode: 1.8910014629364014 seconds

In [47]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DecimalType, DateType, StringType, LongType

start_time = time.time()

table_name = 'income_band'
table_path = db_path + table_name
raw_table_data = raw_data + table_name + ".dat"

# Create Iceberg table
spark.sql(f"""CREATE TABLE IF NOT EXISTS {db}.{table_name}(
    ib_income_band_sk         integer                          ,
    ib_lower_bound            integer                       ,
    ib_upper_bound            integer
)
USING {table_format} 
LOCATION '{table_path}'""");

# Read CSV file
df = (
    spark.read.format("csv").
    option("header", False)
    .option("delimiter", "|")
    .load(raw_table_data)
)
df = df.drop(df.columns[-1])


# Define column names and types
columns_mapping = [
    ("ib_income_band_sk", IntegerType()),
    ("ib_lower_bound", IntegerType()),
    ("ib_upper_bound", IntegerType()),
]

# Rename columns and perform type casting
for col_name, col_type in columns_mapping:
    df = df.withColumnRenamed(f"_c{columns_mapping.index((col_name, col_type))}", col_name)
    df = df.withColumn(col_name, col(col_name).cast(col_type))

# Write to Iceberg table
df.write.format(f"{table_format}").mode("append").option("mergeSchema", "true").saveAsTable(f"{db}.{table_name}")

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time
execution_times[table_name] = execution_time
print(f"Execution time for {table_name}: {execution_time} seconds")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time for income_band: 1.7389919757843018 seconds

In [ ]:
###########################################################################################################################################################################
###########################################################################################################################################################################
###########################################################################################################################################################################

In [63]:


# SELECT statements for the first 10 rows of each table
call_center_query = "SELECT * FROM dev.iceberg11.call_center LIMIT 10"
catalog_page_query = "SELECT * FROM dev.iceberg11.catalog_page LIMIT 10"
catalog_returns_query = "SELECT * FROM dev.iceberg11.catalog_returns LIMIT 10"
catalog_sales_query = "SELECT * FROM dev.iceberg11.catalog_sales LIMIT 10"
customer_query = "SELECT * FROM dev.iceberg11.customer LIMIT 10"
customer_address_query = "SELECT * FROM dev.iceberg11.customer_address LIMIT 10"
customer_demographics_query = "SELECT * FROM dev.iceberg11.customer_demographics LIMIT 10"
date_dim_query = "SELECT * FROM dev.iceberg11.date_dim LIMIT 10"
household_demographics_query = "SELECT * FROM dev.iceberg11.household_demographics LIMIT 10"
income_band_query = "SELECT * FROM dev.iceberg11.income_band LIMIT 10"
inventory_query = "SELECT * FROM dev.iceberg11.inventory LIMIT 10"
item_query = "SELECT * FROM dev.iceberg11.item LIMIT 10"
promotions_query = "SELECT * FROM dev.iceberg11.promotion LIMIT 10"
reason_query = "SELECT * FROM dev.iceberg11.reason LIMIT 10"
ship_mode_query = "SELECT * FROM dev.iceberg11.ship_mode LIMIT 10"
store_query = "SELECT * FROM dev.iceberg11.store LIMIT 10"
store_returns_query = "SELECT * FROM dev.iceberg11.store_returns LIMIT 10"
store_sales_query = "SELECT * FROM dev.iceberg11.store_sales LIMIT 10"
time_dim_query = "SELECT * FROM dev.iceberg11.time_dim LIMIT 10"
warehouse_query = "SELECT * FROM dev.iceberg11.warehouse LIMIT 10"
web_page_query = "SELECT * FROM dev.iceberg11.web_page LIMIT 10"
web_returns_query = "SELECT * FROM dev.iceberg11.web_returns LIMIT 10"
web_sales_query = "SELECT * FROM dev.iceberg11.web_sales LIMIT 10"
web_site_query = "SELECT * FROM dev.iceberg11.web_site LIMIT 10"

# Execute the queries and show the results
call_center_df = spark.sql(call_center_query)
catalog_page_df = spark.sql(catalog_page_query)
catalog_returns_df = spark.sql(catalog_returns_query)
catalog_sales_df = spark.sql(catalog_sales_query)
customer_df = spark.sql(customer_query)
customer_address_df = spark.sql(customer_address_query)
customer_demographics_df = spark.sql(customer_demographics_query)
date_dim_df = spark.sql(date_dim_query)
household_demographics_df = spark.sql(household_demographics_query)
income_band_df = spark.sql(income_band_query)
inventory_df = spark.sql(inventory_query)
item_df = spark.sql(item_query)
promotions_df = spark.sql(promotions_query)
reason_df = spark.sql(reason_query)
ship_mode_df = spark.sql(ship_mode_query)
store_df = spark.sql(store_query)
store_returns_df = spark.sql(store_returns_query)
store_sales_df = spark.sql(store_sales_query)
time_dim_df = spark.sql(time_dim_query)
warehouse_df = spark.sql(warehouse_query)
web_page_df = spark.sql(web_page_query)
web_returns_df = spark.sql(web_returns_query)
web_sales_df = spark.sql(web_sales_query)
web_site_df = spark.sql(web_site_query)

# Show the results
call_center_df.show()
catalog_page_df.show()
catalog_returns_df.show()
catalog_sales_df.show()
customer_df.show()
customer_address_df.show()
customer_demographics_df.show()
date_dim_df.show()
household_demographics_df.show()
income_band_df.show()
inventory_df.show()
item_df.show()
promotions_df.show()
reason_df.show()
ship_mode_df.show()
store_df.show()
store_returns_df.show()
store_sales_df.show()
time_dim_df.show()
warehouse_df.show()
web_page_df.show()
web_returns_df.show()
web_sales_df.show()
web_site_df.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----------------+-----------------+---------------+-----------------+---------------+-------------+--------+------------+--------+--------+--------------+---------+--------------------+--------------------+-----------------+-----------+----------------+----------+---------------+----------------+--------------+--------------+---------------+-------+-----------------+--------+------+-------------+-------------+-----------------+
|cc_call_center_sk|cc_call_center_id|cc_rec_start_date|cc_rec_end_date|cc_closed_date_sk|cc_open_date_sk|      cc_name|cc_class|cc_employees|cc_sq_ft|cc_hours|    cc_manager|cc_mkt_id|        cc_mkt_class|         cc_mkt_desc|cc_market_manager|cc_division|cc_division_name|cc_company|cc_company_name|cc_street_number|cc_street_name|cc_street_type|cc_suite_number|cc_city|        cc_county|cc_state|cc_zip|   cc_country|cc_gmt_offset|cc_tax_percentage|
+-----------------+-----------------+-----------------+---------------+-----------------+-------

In [57]:
spark.sql("SELECT * FROM dev.iceberg11.call_center LIMIT 10").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----------------+-----------------+---------------+-----------------+---------------+-------------+--------+------------+--------+--------+--------------+---------+--------------------+--------------------+-----------------+-----------+----------------+----------+---------------+----------------+--------------+--------------+---------------+-------+-----------------+--------+------+-------------+-------------+-----------------+
|cc_call_center_sk|cc_call_center_id|cc_rec_start_date|cc_rec_end_date|cc_closed_date_sk|cc_open_date_sk|      cc_name|cc_class|cc_employees|cc_sq_ft|cc_hours|    cc_manager|cc_mkt_id|        cc_mkt_class|         cc_mkt_desc|cc_market_manager|cc_division|cc_division_name|cc_company|cc_company_name|cc_street_number|cc_street_name|cc_street_type|cc_suite_number|cc_city|        cc_county|cc_state|cc_zip|   cc_country|cc_gmt_offset|cc_tax_percentage|
+-----------------+-----------------+-----------------+---------------+-----------------+-------

In [ ]:
# Define the S3 path
s3_path = "s3://raw1tcpds/data/web_sales.dat"

# Read the CSV file into a DataFrame
df = spark.read.option("delimiter", "|").csv(s3_path)

# Specify the column name
column_name = "_c3"

# Check for null values in the specified column
null_count = df.filter(col(column_name).isNull()).count()

if null_count > 0:
    print(f"Column '{column_name}' has {null_count} null values.")
else:
    print(f"Column '{column_name}' does not have any null values.")

In [10]:
spark.sql("select * from dev.iceberg11.store_sales").show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+---------------+----------+--------------+-----------+-----------+----------+-----------+-----------+----------------+-----------+-----------------+-------------+--------------+-------------------+------------------+---------------------+-----------------+----------+-------------+-----------+-------------------+-------------+
|ss_sold_date_sk|ss_sold_time_sk|ss_item_sk|ss_customer_sk|ss_cdemo_sk|ss_hdemo_sk|ss_addr_sk|ss_store_sk|ss_promo_sk|ss_ticket_number|ss_quantity|ss_wholesale_cost|ss_list_price|ss_sales_price|ss_ext_discount_amt|ss_ext_sales_price|ss_ext_wholesale_cost|ss_ext_list_price|ss_ext_tax|ss_coupon_amt|ss_net_paid|ss_net_paid_inc_tax|ss_net_profit|
+---------------+---------------+----------+--------------+-----------+-----------+----------+-----------+-----------+----------------+-----------+-----------------+-------------+--------------+-------------------+------------------+---------------------+-----------------+----------+-------------+-----------+

In [64]:
print(execution_times)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'inventory': 20.63095498085022}